In [61]:
# Import necessary packages
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [62]:
reduced_books_users_ratings_locations = pd.read_csv("https://raw.githubusercontent.com/GoldbergData/Machine-Learning-Book-Ratings/master/data/clean/reduced_books_users_ratings_locations.csv")
reduced_books_users_ratings_locations.head(2)

,user_id,isbn,book_rating,book_title,book_author,year_of_publication,publisher,unique_isbn,age,city,state,country
0,16877,038550120X,9,A Painted House,JOHN GRISHAM,2001,Doubleday,038550120X,37.000,houston,arkansas,usa
1,16877,034539657X,7,Dark Rivers of the Heart,Dean R. Koontz,1995,Ballantine Books,034539657X,37.000,houston,arkansas,usa


### Part I: kNN Recommender Algorithm - Books 

#### Combine user_id, unique_isbn, book_rating and book_title into one dataset

In [63]:
#reduced_books_users_ratings = reduced_books_users_ratings.drop(['book_author','year_of_publication','publisher','isbn','age','city','state','country','dropornot','dropuser'], axis = 1)
reduced_books_users_ratings_locations.head(5)

,user_id,isbn,book_rating,book_title,book_author,year_of_publication,publisher,unique_isbn,age,city,state,country
0,16877,038550120X,9,A Painted House,JOHN GRISHAM,2001,Doubleday,038550120X,37.000,houston,arkansas,usa
1,16877,034539657X,7,Dark Rivers of the Heart,Dean R. Koontz,1995,Ballantine Books,034539657X,37.000,houston,arkansas,usa
2,16877,743211383,3,Dreamcatcher,Stephen King,2001,Scribner,61083259,37.000,houston,arkansas,usa
3,16877,786868716,10,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,786868716,37.000,houston,arkansas,usa
4,16877,440159016,10,Motherhood: The Second Oldest Profession,Erma Bombeck,1987,Dell,70064547,37.000,houston,arkansas,usa


#### Create a column for Ratings count and group by Book Title

In [64]:
book_total_rating = (reduced_books_users_ratings_locations.
     groupby(by = ['book_title'])['book_rating'].
     count().
     reset_index().
     rename(columns = {'book_rating': 'total_rating_count'})
     [['book_title', 'total_rating_count']]
    )
book_total_rating.head()

,book_title,total_rating_count
0,'Salem's Lot,19
1,10 Lb. Penalty,15
2,100 Selected Poems by E. E. Cummings,6
3,101 Dalmatians,9
4,11-Sep,13


#### Combine both datasets

In [65]:
book_rating_count_user = reduced_books_users_ratings_locations.merge(book_total_rating, left_on = 'book_title', right_on = 'book_title', how = 'left')
book_rating_count_user.head()

,user_id,isbn,book_rating,book_title,book_author,year_of_publication,publisher,unique_isbn,age,city,state,country,total_rating_count
0,16877,038550120X,9,A Painted House,JOHN GRISHAM,2001,Doubleday,038550120X,37.000,houston,arkansas,usa,237
1,16877,034539657X,7,Dark Rivers of the Heart,Dean R. Koontz,1995,Ballantine Books,034539657X,37.000,houston,arkansas,usa,32
2,16877,743211383,3,Dreamcatcher,Stephen King,2001,Scribner,61083259,37.000,houston,arkansas,usa,150
3,16877,786868716,10,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,786868716,37.000,houston,arkansas,usa,168
4,16877,440159016,10,Motherhood: The Second Oldest Profession,Erma Bombeck,1987,Dell,70064547,37.000,houston,arkansas,usa,6


In [66]:
book_rating_count_user = book_rating_count_user.drop(['isbn','book_author','year_of_publication','publisher','age','city','state','country'], axis = 1)
book_rating_count_user.head()

,user_id,book_rating,book_title,unique_isbn,total_rating_count
0,16877,9,A Painted House,038550120X,237
1,16877,7,Dark Rivers of the Heart,034539657X,32
2,16877,3,Dreamcatcher,61083259,150
3,16877,10,The Five People You Meet in Heaven,786868716,168
4,16877,10,Motherhood: The Second Oldest Profession,70064547,6


#### Look at statistics of the total ratings count

In [67]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(book_total_rating['total_rating_count'].describe())

count   5580.000
mean      20.843
std       25.542
min        1.000
25%        9.000
50%       13.000
75%       22.000
max      431.000
Name: total_rating_count, dtype: float64


In [68]:
print(book_total_rating['total_rating_count'].quantile(np.arange(.9, 1, .01)))

0.900    41.000
0.910    44.000
0.920    48.000
0.930    52.000
0.940    57.000
0.950    62.000
0.960    71.000
0.970    83.000
0.980    99.420
0.990   135.210
Name: total_rating_count, dtype: float64


In [69]:
popularity_threshold = 135 # to ensure statistical significance ==> top 5 % 
rating_by_popularity = book_rating_count_user.query('total_rating_count >= @popularity_threshold')
print(rating_by_popularity.shape)
rating_by_popularity.head()

(10976, 5)


,user_id,book_rating,book_title,unique_isbn,total_rating_count
0,16877,9,A Painted House,038550120X,237
2,16877,3,Dreamcatcher,61083259,150
3,16877,10,The Five People You Meet in Heaven,786868716,168
6,20806,6,A Painted House,038550120X,237
10,21340,9,A Painted House,038550120X,237


#### Implement Nearest Neighbors - Brute Force algorithm
* Transform the Rating values of the matrix dataframe into a scipy sparse matrix for more efficient calculations.
* The algorithm will calculate the cosine similarity between rating vectors using cosine metric and brute algorithm

Neighbors-based classification is a type of instance-based learning or non-generalizing learning: it does not attempt to construct a general internal model, but simply stores instances of the training data. Classification is computed from a simple majority vote of the nearest neighbors of each point: a query point is assigned the data class which has the most representatives within the nearest neighbors of the point.

In [70]:
# Drop duplicates
rating_by_popularity = rating_by_popularity.drop_duplicates(['user_id', 'book_title'])

# Fill missing values with 0 for vector calculation
rating_by_popularity_pivot = rating_by_popularity.pivot(index = 'book_title', columns = 'user_id', values = 'book_rating').fillna(0)

# Create a scipy sparse matrix nd convert data into Compressed Sparse Row format
rating_by_popularity_matrix = csr_matrix(rating_by_popularity_pivot.values)

# Initialize the model, implement NearestNeighbors unsupervised learner 
knn_nn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

# Fit the model
knn_nn.fit(rating_by_popularity_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [82]:
rating_by_popularity_pivot.index[query_index]

"Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))"

In [72]:
distances

array([[0.        , 0.36073555, 0.42312316, 0.53774089, 0.58405745,
        0.74287496]])

In [73]:
knn_nn.kneighbors_graph(rating_by_popularity_matrix).toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

The dataset is structured such that points nearby in index order are nearby in parameter space, leading to an approximately block-diagonal matrix of K-nearest neighbors.

#### Test the model's performance and make recommendations
kNN calculates the Euclidian distance to measure how close observations are to each other 

In [83]:
query_index = np.random.choice(rating_by_popularity_pivot.shape[0])

# Try with 6 nearest neighbors. Use iloc to locate observations that match the query_index
distances,indices = knn_nn.kneighbors(rating_by_popularity_pivot.iloc[query_index,:].values.reshape(1,-1),n_neighbors = 6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(rating_by_popularity_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, rating_by_popularity_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for 1st to Die: A Novel:

1: The Summons, with distance of 0.8904265254691257:
2: The Partner, with distance of 0.9158488838696803:
3: The Chamber, with distance of 0.9167916490690231:
4: The Secret Life of Bees, with distance of 0.9172396093918462:
5: Suzanne's Diary for Nicholas, with distance of 0.9207408036249799:


#### Cross - Validation
* Because kNN does not have traditional training and testing phases as it uses the entire dataset for training and then calculates nearest neighbors for each new observation it is hard to measure its accruacy.
* We are going to use k-fold and LOOCV for validation that automatically split the entire dataset into train and test

In [80]:
# K-fold Cross-Validation
kf = KFold(n_splits=2)
for train, test in kf.split(rating_by_popularity_matrix):
    print("%s %s" % (train, test))

[29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
 53 54 55 56 57] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28] [29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
 53 54 55 56 57]


In [81]:
# LOO Cross-Validation
loo = LeaveOneOut()
for train, test in loo.split(rating_by_popularity_matrix):
    print("%s %s" % (train, test))

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57] [0]
[ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57] [1]
[ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57] [2]
[ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57] [3]
[ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57] [4]
[ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 3